In [ ]:
import numpy as np

In [ ]:
positions = {}

with open("input", "r") as f:
    positions["p1"] = int(f.readline().split(":")[1].strip()) - 1
    positions["p2"] = int(f.readline().split(":")[1].strip()) - 1
    
positions2 = np.array(list(positions.values()))

### Part 1

In [ ]:
def deterministic_roll():
    n = 0
    while True:
        yield n % 100 + 1
        n += 1

In [ ]:
roll = deterministic_roll()

scores = {"p1": 0, "p2": 0}
throws = 0

while scores["p1"] < 1000 and scores["p2"] < 1000:
    for p in ("p1", "p2"):
        throw = sum(next(roll) for i in range(3))
        throws += 3
        positions[p] = (positions[p] + throw) % 10
        scores[p] += positions[p] + 1
        if scores[p] >= 1000:
            break
            
min(scores.values()) * throws

### Part 2

In [ ]:
cache = {}

def dirac_roll(positions, scores, player):
    
    cache_key = (tuple(positions), tuple(scores), player)
    
    if cache_key in cache:
        return cache[cache_key]
    
    throws = np.tensordot(np.sum(np.meshgrid(*np.tile(np.arange(1, 4), 3).reshape((3, 3))),
                                 axis=0, dtype=int).flatten(),
                          np.eye(1, 2, player, dtype=int).flatten(), axes=0)
    
    new_pos = (throws + positions) % 10
    
    new_scores = scores + (new_pos + 1) * np.eye(1, 2, player, dtype=int).flatten()
    next_player = (player + 1) % 2

    score_filter = new_scores >= 21
        
    wins = new_scores[score_filter[:, player], :].shape[0]
    win_score = np.eye(1, 2, player, dtype=int) * wins

    undecided = new_scores[~score_filter[:, player], :]
    undecided_pos = new_pos[~score_filter[:, player], :]
    
    if undecided.shape[0] == 0:
        cache[cache_key] = win_score
        return win_score
    
    cache[cache_key] = np.sum(np.vstack((win_score,
                                         dirac_roll(undecided_pos, undecided, next_player))),
                              axis=0, dtype=int)

    return cache[cache_key]

dirac_roll = np.vectorize(dirac_roll, signature='(m),(m),()->(m)')

In [ ]:
np.max(dirac_roll(positions2, np.zeros(2), 0))